In [4]:
import json
from rouge import Rouge
import os
import pandas as pd
import numpy as np
import copy
import jieba
import sys
import re
import logging
import gensim
from gensim.models.word2vec import LineSentence
from sklearn import model_selection
from config import config_base
import vocab
from preprocess_data import *
config = config_base.config

In [8]:
df = organize_data(config.train_data)
df = df.sample(5000)

In [9]:
df = deal_data(df)

In [10]:
df = shorten_content_all(df, config.max_len)

shorten content, accuracy: 0.9664
max length: 300
min length: 7
mean length:180
median length:176


In [239]:
s = build_answer_range(df)

[0.0, 0.125, 0.0, 0.125, 0.125]
merge 美媒盘点世界十大导弹。据美国军事技术网（Army-technology）报道，洲际弹道导弹（ICBM）被认为是一种主要的核威慑力量，目前只有少数国家拥有发射这种远距离武器的技术。本文以射程为参考，列出排名前十的洲际弹道导弹。 No.1 R-36M（SS-18撒旦） 俄罗斯 射程：16000公里 图为 R-36M（SS-18撒旦）洲际弹道导弹 R-36M（SS-18撒旦）是世界上射程最远的洲际弹道导弹，射程范围为16000公里。同时，SS-18以其8.8吨的发射重量，成为世界上最重的洲际弹道导弹。 2013-11-26 新华网
answer 洲际弹道导弹
question 什么是一种主要的核威慑力量？
[0.6666666666666666, 0.6666666666666666, 0.0, 0.0, 0.0, 0.0]
merge 习近平谈马克思主义。习近平谈马克思主义 新华网北京5月4日电 习近平今天上午在纪念马克思诞辰200周年大会上说，马克思主义是科学的理论，创造性地揭示了人类社会发展规律；马克思主义是人民的理论，第一次创立了人民实现自身解放的思想体系；马克思主义是实践的理论，指引着人民改造世界的行动；马克思主义是不断发展的开放的理论，始终站在时代前沿。(据新华社“新华视点”微博报道)
answer 马克思主义
question 习近平谈什么
[0.6, 0.6, 0.1, 0.1]
merge 中华人民共和国和奥地利共和国关于建立友好战略伙伴关系的联合声明。新华社北京4月8日电 中华人民共和国和奥地利共和国关于建立友好战略伙伴关系的联合声明 应中华人民共和国主席习近平邀请，奥地利共和国总统亚历山大范德贝伦于2018年4月7日至13日对中华人民共和国进行国事访问。奥地利共和国总理塞巴斯蒂安库尔茨和多位部长陪同访华
answer 奥地利共和国
question 中华人民共和国和谁关于建立友好战略伙伴关系的联合声明
[0.6153846153846154, 0.6153846153846154, 0.15384615384615385, 0.07692307692307693, 0.0]
merge 俄罗斯已将军队部署叙利亚 称当地“完全稳定”。俄罗斯已将军队部署叙利亚 称当地“完全稳定”

[0.16666666666666666, 0.16666666666666666, 0.08333333333333333, 0.125]
merge 伊朗：与欧盟就伊核协议问题进行建设性会谈。（早报讯） 据伊朗塔斯尼姆通讯社15日报道，伊朗外长扎里夫当天在布鲁塞尔表示，伊朗与欧盟就伊核协议问题进行了建设性会谈。报道说，扎里夫在与欧盟外交和安全政策高级代表莫盖里尼会谈后表示，这是一次良好的、建设性的对话，“我们已经在着手达成保障性措施”。扎里夫表示，除美国外，目前伊核协议其他各方在确保共同利益，特别是伊朗利益方面“处在正确轨道上”。扎里夫13日开始接连访问中国、俄罗斯和部分欧洲国家，就美国退出伊核协议后的一系列问题与其他各方进行磋商。美国总统特朗普本月8日宣布，美国退出伊核协议，并重启因伊核协议而豁免的对伊朗制裁。伊朗方面表示，将暂时留在伊核协议中，并与协议其他各方磋商
answer 美国
question 除什么国家之外，目前伊核协议其他各方在确保共同利益，特别是伊朗利益方面“处在正确轨道上”？
[0.2857142857142857, 0.2857142857142857]
merge 王毅同日本外相会谈：共同推动中日关系尽早重回正轨。据外交部网站消息，2018年1月28日，外交部长王毅在北京钓鱼台国宾馆同到访的日本外相河野太郎举行会谈。王毅欢迎河野外相访华，表示今年是中日和平友好条约缔结40周年，对两国关系具有特殊和重要意义。日本外相河野太郎年初即来华访问，体现了日本政府希望改善两国关系的强烈愿望，中方对此表示肯定。期待此访取得积极成果。王毅说，改善和发展中日关系符合两国利益，也是两国社会各界的共同愿望。当前，中日关系处于一个关键阶段，既有积极进展，同时也面临不少干扰和阻碍。希望日方不松劲，不后退，把口头表态转化为实实在在的行动，与中方共同努力，相向而行，推动两国关系尽早回到正常和健康发展的轨道
answer 河野太郎
question 日本外相到你这是什么
[0.4, 0.4, 0.4, 0.1, 0.1, 0.1, 0.1]
merge 局面不可接受？伊朗不要“没有好处”的核协议。局面不可接受？伊朗不要“没有好处”的核协议 伊朗副外长阿巴斯？阿拉格希22日表示，如果伊核协议不能为伊朗带来经济利益，伊朗将考虑退出协议。此前，美国总统唐纳德？特朗普要求修改协

[0.4444444444444444, 0.4444444444444444]
merge 驻英美军开枪阻止男子驾车闯基地 无关恐怖主义。驻英美军开枪阻止男子驾车闯基地 无关恐怖主义 英国一名男子18日驾车硬闯一处美国驻英空军基地，遭美军开枪拦截后被捕。当地警方认定，这起事件无关恐怖主义。 米尔登霍尔空军基地证实，当天接到正常秩序受“扰乱”的报告后，基地下午13时左右进入封闭状态，大约一个半小时后解除封闭。 “美国军人开枪射击，一名男子被扣押，身上有擦伤和瘀伤，”这座军事基地所处萨福克郡的警方说，“没有其他人员受伤。(陈丹)【新华社微特稿】
answer 驻英美军
question 谁开枪阻止男子驾车闯基地 无关恐怖主义
[0.45454545454545453, 0.45454545454545453]
merge 历时245天，中国海军第26批护航编队凯旋。由东海舰队导弹护卫舰黄冈舰、扬州舰，综合补给舰高邮湖舰组成的第26批护航编队，在圆满完成亚丁湾护航和访问欧洲4国任务后，于12月1日返回舟山某军港。 历时245天，累计航程14.5万余海里―― 我第26批护航编队凯旋 由东海舰队导弹护卫舰黄冈舰、扬州舰，综合补给舰高邮湖舰组成的第26批护航编队，在圆满完成亚丁湾护航和访问欧洲4国任务后，于12月1日返回舟山某军港。东海舰队在码头举行欢迎仪式。 自4月1日从舟山起航后，编队历时245天，累计航程14.5万余海里，共完成为联合国世界粮食计划署船舶护航等42批护航任务，续写了被护船舶和编队自身“两个百分之百安全”纪录。 完成护航任务后，编队先后访问了比利时、丹麦、英国、法国。（特约记者代宗锋）
answer 东海舰队导弹护卫舰黄冈舰、扬州舰，综合补给舰高邮湖舰
question 由哪些舰船组成的第26批护航编队？
[0.4, 0.4, 0.2, 0.1, 0.3, 0.1]
merge 媒体：阿富汗在美军撤走后或将陷入混乱。媒体：阿富汗在美军撤走后或将陷入混乱 《华盛顿邮报》周日援引为指名的美国官员的话报道，“塔利班”武装分子在今后几年将在阿富汗巩固自己的阵地。 美国官员阅览了情报部有关塔利班在阿富汗的影响力将会增长的报告。报刊认为，甚至在2014年后在阿富汗将留下数千名美军以及华盛顿将继续资助政府的条件下，塔利班的影响力还是继续增长。 情报部门的报告中说，如果华

[0.42857142857142855, 0.42857142857142855]
merge 美军开发"呼叫空袭"手机应用程序。核心提示： 美军正开发一种应用程序，有朝一日，战斗中的军人也许可以用智能手机叫来轰炸机实施空袭。参考消息网10月23日报道 美媒称，美国军方正在开发一种应用程序，有朝一日，战斗中的军人也许可以用智能手机叫来轰炸机实施空袭。美国趣味科学网站10月21日报道称， 据该程序的开发者说，这一技术可能有助于在作战行动中降低误伤友军或平民的可能性。这种应用程序名叫ATAK（安卓终端攻击工具），是由位于马萨诸塞州剑桥的一家非营利性质的研发实验室德雷珀实验室开发的。据该实验室的负责人说，这种系统将适用于安卓手机，也可用于导航、空间认知。2013-10-23 参考消息网
answer 智能手机
question 美国军方正在开发一种应用程序，有朝一日，战斗中的军人也许可以用什么叫来轰炸机实施空袭
[0.11666666666666667, 0.11666666666666667]
merge 喀麦隆边境地区发生两次袭击4死3失踪。喀麦隆极北大区边境地带11日发生两次袭击，共造成4人死亡，另有3人失踪。当地消息人士向新华社记者证实，当地时间11日5时45分左右，一名疑似极端组织“博科圣地”的自杀式袭击者在靠近尼日利亚边境的喀麦隆凯拉瓦的一座清真寺内引爆随身爆炸物，造成2名平民死亡，袭击者当场毙命。另一起袭击事件发生在当地时间11日10时左右，一名卡车司机在喀麦隆1号国道靠近尼日利亚边境处行驶时遭疑似“博科圣地”成员枪击身亡，车上3名乘客目前下落不明。“博科圣地”组织成立10余年来，在尼日利亚及喀麦隆等国频繁制造恐怖袭击，导致大量人员伤亡。（记者乔本孝）
answer 4
question 喀麦隆极北大区边境地带11日发生两次袭击，共造成几人死亡，另有3人失踪。 当地消息人士向新华社记者证实，当地时间11日5时45分左右，一名疑似极端组织“博科圣地”的自杀式袭击者在靠近尼日利亚边境的喀麦隆凯拉瓦的一座清真寺内引爆随身爆炸物，造成2名平民死亡，袭击者当场毙命。
[0.05, 0.05, 0.2, 0.2, 0.1]
merge 习近平同越南国家主席陈大光举行会谈。中共中央总书记、国家主席习近平13日在河内主席府同越南国家主席陈大光举行会谈。习近平祝贺越南社会主

[0.23809523809523808, 0.0, 0.23809523809523808]
merge 联合国副秘书长费尔特曼抵达朝鲜。图为费尔特曼(中)及2名随行人员在平壤顺安国际机场的贵宾通道合影。(莽九晨 摄)【环球时报驻朝鲜特派记者 莽九晨 环球时报-环球网记者 姚丽娟】当地时间5日下午4时许，联合国负责政治事务的副秘书长杰弗里？费尔特曼抵达朝鲜。朝鲜外务省国际机构局副局长申东国(音译)前往机场迎接。申东国在机场迎接时对费尔特曼一行表示欢迎，费尔特曼则向申东国表示感谢。联合国方面此前介绍说，费尔特曼将与朝方官员就“共同关注的问题”进行讨论。5日下午，费尔特曼一行在机场没有接受任何采访，也没有发表任何讲话
answer 朝鲜
question 当地时间5日下午4时许，联合国负责政治事务的副秘书长杰弗里？费尔特曼抵达哪里
[0.18604651162790697, 0.18604651162790697]
merge 外交部：愿同印方一道推动中印关系沿着正确轨道向前发展。国际在线报道（记者 兰旻）：对于印度外交部有关中印关系的表态，中国外交部发言人陆慷12日在北京表示，中方愿同印方一道，以两国领导人重要共识为指引，增进政治互信，推进互利合作，妥善管控分歧，推动两国关系沿着正确轨道向前发展。 近日，印度外交部发言人称，印中关系对两国、本地区和全球至关重要。印方愿同中方基于共同点发展双边关系，并在相互尊重和对彼此利益、关切和愿望保持敏感的基础上处理分歧。在当天的例行记者会上，陆慷就此表示，“我们注意到印方有关表态。” 标签：
answer 中印关系
question 对于印度外交部有关什么的表态，中国外交部发言人陆慷12日在北京表示，中方愿同印方一道，以两国领导人重要共识为指引，增进政治互信，推进互利合作，妥善管控分歧，推动两国关系沿着正确轨道向前发展。
[0.15151515151515152, 0.15151515151515152]
merge 韩国总统文在寅下令部署剩余4辆“萨德”发射车。人民网首尔7月29日电（夏雪）韩联社援引韩国总统府青瓦台29日消息称，韩国总统文在寅当天凌晨主持召开国家安全保障会议（NSC）全体会议，指示韩军进行与美军联合发射弹道导弹等对朝武力示威，并下令部署剩余4辆“萨德”发射车。该报道称，此外，文在寅要求有关部门立即与美方讨论加

question 把思政课讲得“什么
[0.1111111111111111, 0.1111111111111111]
merge 马蒂斯应对APP泄密：美军或禁用个人定位设备（6）。图为Shank基地内一座孤零零的岗楼，这里曾是北约联军重兵驻防的地区，现在部队已撤走，大批营房也已拆除，只剩下一些遗迹，这座基地将会移交给阿富汗国民军，摄于2014年4月3日。 图为遗留在Shank基地内的标示，或许曾经是一个战地洗消点，标示要求通过该区域的人要穿戴防护服、面罩和眼罩
answer Shank基地
question 哪里曾是北约联军重兵驻防的地区
[0.7272727272727273, 0.7272727272727273, 0.09090909090909091]
merge 美军两架直升机坠毁：狂风巨浪 12名军人仍失踪。原标题：美军两架直升机坠毁：狂风巨浪 12名军人仍失踪 CH-53K直升机。 参考消息网1月18日报道？美联社1月17日报道称，救援人员16日迎着时速达37公里的大风和高达9米的巨浪继续搜救失踪的12名海军陆战队员。他们乘坐的两架直升机早前在夏威夷瓦胡岛附近海域坠毁。 大风大浪冲散了飞机残骸，增加了搜救难度。 CH-53E直升机也被称为“超级种马”运输直升机，是美军现役最大的直升机，可以运输一辆轻型装甲车，或者16吨货物，或者一支装备完整的海军陆战队队伍
answer 12名
question 美军两架直升机坠毁：狂风巨浪 多少名军人仍失踪
[0.35714285714285715, 0.35714285714285715]
merge 国际社会吁就巴以新冲突展开独立调查 遭以方拒绝。原标题：国际社会吁就巴以新冲突展开独立调查 遭以方拒绝 瑞士资讯4月1日援引法新社报道称，以色列士兵3月30日在沿加沙走廊边界的示威冲突中打死16名巴勒斯坦人，打伤数百人。国际社会针对此案进行独立调查的要求，在当地时间4月1日遭到以色列拒绝。 据报道，人权团体质疑以色列军为何以实弹进行攻击。巴勒斯坦人则指控以士兵对不具威胁性的示威者开火。他在1日重申反对进行独立调查
answer 以方
question 国际社会吁就巴以新冲突展开独立调查 遭哪方拒绝？
[0.2, 0.2]
merge 国防部就日方炒作中国舰艇进入钓鱼岛毗连区问题答问。新华社北京1月11日电 国防部

[0.5, 0.5]
merge 马警逮捕八恐嫌 一人涉策划袭击宗教场所。八名分别在沙巴、雪兰莪和霹雳州落网的恐嫌中，四名是马国人、三名菲律宾人及一名阿尔巴尼亚人。该阿尔巴尼亚籍男子是大学客座讲师，相信与伊国组织有联系。 马来西亚警察总长弗兹昨天发表文告指出，八名恐嫌当中有四名马来西亚人、三名菲律宾人及一名阿尔巴尼亚人；他们是在上个月27日至本月6日期间，分别在沙巴、雪兰莪和霹雳州落网。 警方于上周三（27日）先在沙州山打根逮捕两名马来西亚人和三名菲律宾人，其中一名菲律宾人是马来西亚永久居民。 两恐嫌狱中募新血 另两名马来西亚籍嫌犯则是前天在霹州打巴落网。 他们曾因涉及恐怖活动而在内安法令及防范罪案法令下被扣留七年，去年1月获释后继续参与恐怖活动。 他指出，警方是在8月30日，也就是国庆日前夕，逮捕八名阿布沙耶夫分子而掌握了可靠的情报和线索，于是在9月份接连展开多次突击行动，扫荡匿藏在马来西亚各地的恐怖分子
answer 沙巴、雪兰莪和霹雳州
question 恐怖分子在哪里落网
[0.875, 0.875, 0.25]
merge 菲律宾总统杜特尔特就香港人质事件正式道歉。菲律宾总统杜特尔特 资料图。 中新社记者 关向东 摄 菲律宾总统杜特尔特就香港人质事件正式道歉 4月12日晚，菲律宾总统杜特尔特在香港会见菲律宾劳工的时候，就2010年发生在菲律宾的香港人质事件道歉。 杜特尔特说：“请允许我对中国人民表示，从我的内心深处，我作为菲律宾的总统，谨代表菲律宾人民，就此向你们正式道歉。” 这是菲律宾在任总统首次就此事进行正式道歉。 杜特尔特说：“我们对发生的意外深表遗憾。(央视记者 黄铮铮)
answer 香港人质事件
question 菲律宾总统杜特尔特就什么正式道歉
[0.46153846153846156, 0.46153846153846156]
merge 每日快讯 (2005/04/21)。每 日 快 讯2005/04/21 1. 美报告称应尽快研制其远程攻击系统2. 定向能武器发展减缓 3. 洛克希德？马丁公司继续研制分布式交互指挥部件4. 美国空军确定未来工作重点5. 美派20军官参加"汉光-21"演习 强化与台军合作 6. 台湾考虑购买新一代作战飞机 7. 俄制直升机配装外国发动机成为标准模式8. 德国向印度提供先进常规潜艇详细报道1.美报告称应尽快

[0.7, 0.1, 0.7]
merge 美军讨论分散化网络空间作战指挥 控制战力统一。导读：在有关网络的讨论中，战区网络经常被忽视，因为通常强调稀有的、高端的秘密打击能力和关键基础设施保护面临的更广泛的挑战。不过，在地理性作战司令部内部，国防部可以迅速地发展网络能力，并且把网络能力集成到其他军事行动。美国网络司令部必须为战区网络建立一个指挥控制结构，确保美国网络司令部有足够的能力在全球范围内监督和管理网络作战，保障作战指挥官按照能力需求获得相应的网络作战能力。如果对网络能力的控制太松的话，那带来的风险可能是运用不当，会产生潜在的战略影响。在有关网络的讨论中，战区网络经常被忽视，因为通常强调稀有的、高端的秘密打击能力和关键基础设施保护面临的更广泛的挑战。安德万2015-01-02 知远防务网站
answer 战区网络
question 在有关网络的讨论中，什么经常被忽视
[0.5714285714285714, 0.5714285714285714]
merge 美国海军寻求将云计算加入CANES。美国海军近日透露，统一海上网络和企业服务（CANES）项目办公室与海军研究局（ONR）正合作试验，希望利用云计算技术减少舰载系统的硬件数量。云计算技术计划于2018财年完成部署，该技术是CANES项目的基本要求之一。CANES显著降低了海军赛博安全的脆弱性，实现了集中式的赛博防护作战并同时提供了更加鲁棒性的赛博态势感知能力，从而建立了一种新型的信息优势作战态势，这对海军维持海上信息优势能力至关重要。CANES将成为超过200种应用程序和互联系统的网络平台，提供数据、传输、系统管理、话音和视频等服务。海军还将在CANES中加入旧系统中缺少的监视、管理、识别、防护、防御和响应赛博威胁的鲁棒性手段。CANES为海军提供了一种由鲁棒性软件工具集组成的系统管理方法，从而在未来更好地应对赛博威胁。海军于本周分别授予了BAE系统、通用动力、全球技术系统、诺斯罗普格鲁曼和信佳（SERCO）等5家公司一份不定交付日期、不定数量、固定总价、成本加固定酬金形式的CANES组件生产合同，合同总价值约为25.3亿美元。忻欣 译，谭绍杰 校，译自Signal杂志2014-8-21
answer 美国海军
question 谁寻求将云计算加入CANES
[0.3076923076923077, 0.

In [237]:
191/5000

0.0382

In [212]:
tmp = s.sample()

In [154]:
merge = tmp['merge'].values[0]
merge

'俄罗斯红场阅兵武器展示规模缩水体现真实战力。[《新京报》报道] 俄红场阅兵主打爱国牌举行纪念卫国战争胜利66周年阅兵式，两万军人受阅，伊斯坎德尔导弹等重型武器亮相9日上午，俄罗斯举行纪念卫国战争66周年阅兵式，总统梅德韦杰夫检阅了两万多名士兵和多种现代化重型武器装备。 规模缩水展现真实力相对于去年65周年规模空前的盛大阅兵式，66周年的阅兵式规模有所缩水。据俄罗斯媒体介绍，今年红场阅兵的参阅人数比去年有所增加，去年参加红场大阅兵的有一万名军人，但今年参阅人数达到两万。虽然人数增加，但参加检阅的武器装备比去年减少了三成。据俄罗斯媒体介绍，去年参与检阅的有83辆坦克、27套导弹发射系统，36套火炮装置。但今年参与检阅的仅有39辆坦克、18套导弹发射系统，31套火炮装置。2011-5-12 新浪网'

In [155]:
answer = tmp['answer'].values[0]
answer

'两万'

In [156]:
question = tmp['question'].values[0]
question

'今年红场阅兵的参阅人数比去年有所增加，去年参加红场大阅兵的有一万名军人，但今年参阅人数达到多少'

In [157]:
answer_start = tmp['answer_start'].values[0]
answer_start

33.0

In [158]:
answer_end = tmp['answer_end'].values[0]
answer_end

33.0

In [159]:
jieba.lcut(merge)[int(answer_start)-1: int(answer_end+1)+2]

['，', '两万', '军人', '受阅']

In [161]:
sys.setrecursionlimit(1000000)
import rouge
rouge = Rouge(metrics=['rouge-l'])

In [238]:
c = 0
def build_answer_range(df):
    def match(merge, answer, question):
        merge_list = jieba.lcut(merge)
        merge_len = len(merge_list)
        answer_list = jieba.lcut(answer)
        answer_len = len(answer_list)
        question_str = ' '.join(jieba.cut(question))
        start = []
        end = []
        for i in range(0, merge_len-answer_len+1):
            if merge_list[i: i+answer_len] == answer_list:
                start.append(i)
                end.append(i+answer_len-1)
        if len(start) == 0:
            return -1, -1, 0
        elif len(start) == 1:
            return start[0], end[0], 0
        else:
            scores = []
            for s, e in zip(start, end):

                if s >= 5:
                    answer_can = merge_list[s-5: e+5]
                
                elif s== 4:
                    answer_can = merge_list[s-4: e+5]
                elif s == 3:
                    answer_can = merge_list[s-3: e+5]

                elif s == 2:
                    answer_can = merge_list[s-2: e+5]
                elif s == 1:
                    answer_can = merge_list[s-1: e+5]
                else:
                    answer_can = merge_list[s: e+5]
                answer_can = ' '.join(answer_can)
                score = rouge.get_scores(answer_can, question_str, avg=True)['rouge-l']['r']
                scores.append(score)
                

            max_idx = np.argmax(scores)
            max_score = max(scores)
            q = 0
            m = 0
            for i in scores:
                if i == max_score:
                    q+=1
            if q >= 2:
                print(scores)
                print('merge', merge)
                print('answer', answer)
                print('question', question)
                m = 1
                
                
                
            return start[max_idx], end[max_idx], m
            
            
            


    merges = df[df['is_in']]['merge'].values
    answers = df[df['is_in']]['answer'].values
    questions = df[df['is_in']]['question'].values
    answer_range = [match(m, a, q) for m, a, q in zip(merges, answers, questions)]

    start, end, m = list(zip(*answer_range))
    print('m', sum(m))
    df.loc[df['is_in'], 'answer_start'] = start
    df.loc[df['is_in'], 'answer_end'] = end

    merge_len = len(merges)
    right_len = (df['answer_start'] > -1).sum()
    print('answer generation accuracy: %.4f\n' % (right_len/merge_len))

    return df